In [285]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from tqdm import tqdm


In [286]:
df = pd.read_csv('Data/centalpark_from2000.csv')

df.head()

,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,...,WT09,WT11,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT22
0,USW00094728,"NY CITY CENTRAL PARK, NY US",2000-01-01,5.82,0.0,0.0,0.0,42.0,50,34,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094728,"NY CITY CENTRAL PARK, NY US",2000-01-02,4.70,0.0,0.0,0.0,52.0,60,43,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00094728,"NY CITY CENTRAL PARK, NY US",2000-01-03,6.71,0.0,0.0,0.0,58.0,64,51,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094728,"NY CITY CENTRAL PARK, NY US",2000-01-04,7.83,0.7,0.0,0.0,53.0,60,46,...,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,USW00094728,"NY CITY CENTRAL PARK, NY US",2000-01-05,11.41,0.0,0.0,0.0,38.0,47,29,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [287]:
df = df[['DATE', 'TMAX', 'TMIN']]
df.head()

,DATE,TMAX,TMIN
0,2000-01-01,50,34
1,2000-01-02,60,43
2,2000-01-03,64,51
3,2000-01-04,60,46
4,2000-01-05,47,29


In [288]:
df.isnull().sum()

DATE    0
TMAX    0
TMIN    0
dtype: int64

In [289]:
# # Historical TMAX features

# df['TMAX_prev7'] = df['TMAX'].rolling(7, closed = 'left').max()
# df['TMAX_prev3'] = df['TMAX'].rolling(3, closed = 'left').max()
# df['TMAX_prev2'] = df['TMAX'].rolling(2, closed = 'left').max()

# df['TMAX_prev1'] = df['TMAX'].rolling(1, closed = 'left').max()



In [290]:
# # Historical TMIN features

# # df['TMIN_prev7'] = df['TMIN'].rolling(7, closed = 'left').max()
# df['TMIN_prev3'] = df['TMIN'].rolling(3, closed = 'left').max()
# df['TMIN_prev2'] = df['TMIN'].rolling(2, closed = 'left').max()
# df['TMIN_prev1'] = df['TMIN'].rolling(1, closed = 'left').max()


In [291]:
# Historical AWND features

# df['AWND_prev7'] = df['AWND'].rolling(7, closed = 'left').mean()
# df['AWND_prev3'] = df['AWND'].rolling(3, closed = 'left').mean()
# df['AWND_prev1'] = df['AWND'].rolling(1, closed = 'left').sum()


In [292]:
# Historical PRCP features

# df['PRCP_prev7'] = df['PRCP'].rolling(7, closed = 'left').mean()
# df['PRCP_prev3'] = df['PRCP'].rolling(3, closed = 'left').mean()
# df['PRCP_prev1'] = df['PRCP'].rolling(1, closed = 'left').sum()


In [293]:
# df.drop([ 'DATE',], axis = 1, inplace = True)

In [294]:
# Historical TMAX features

df['TMAX_prev10'] = df['TMAX'].rolling(10, closed = 'left').max()
df['TMAX_prev7'] = df['TMAX'].rolling(7, closed = 'left').max()
df['TMAX_prev3'] = df['TMAX'].rolling(3, closed = 'left').max()
df['TMAX_prev2'] = df['TMAX'].rolling(2, closed = 'left').max()
df['TMAX_prev1'] = df['TMAX'].rolling(1, closed = 'left').max()

# Historical TMIN features

#df['TMIN_prev7'] = df['TMIN'].rolling(7, closed = 'left').max()
df['TMIN_prev3'] = df['TMIN'].rolling(3, closed = 'left').max()
df['TMIN_prev2'] = df['TMIN'].rolling(2, closed = 'left').max()
df['TMIN_prev1'] = df['TMIN'].rolling(1, closed = 'left').max()


In [295]:
df.head()

,DATE,TMAX,TMIN,TMAX_prev10,TMAX_prev7,TMAX_prev3,TMAX_prev2,TMAX_prev1,TMIN_prev3,TMIN_prev2,TMIN_prev1
0,2000-01-01,50,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-02,60,43,NaN,NaN,NaN,NaN,50.0,NaN,NaN,34.0
2,2000-01-03,64,51,NaN,NaN,NaN,60.0,60.0,NaN,43.0,43.0
3,2000-01-04,60,46,NaN,NaN,64.0,64.0,64.0,51.0,51.0,51.0
4,2000-01-05,47,29,NaN,NaN,64.0,64.0,60.0,51.0,51.0,46.0


In [296]:
# Removing those buffer rows we used to computer the above features

df = df.loc[10:, :]
df.head()

,DATE,TMAX,TMIN,TMAX_prev10,TMAX_prev7,TMAX_prev3,TMAX_prev2,TMAX_prev1,TMIN_prev3,TMIN_prev2,TMIN_prev1
10,2000-01-11,52,40,64.0,60.0,58.0,58.0,58.0,44.0,44.0,44.0
11,2000-01-12,47,39,64.0,58.0,58.0,58.0,52.0,44.0,44.0,40.0
12,2000-01-13,39,18,64.0,58.0,58.0,52.0,47.0,44.0,40.0,39.0
13,2000-01-14,22,13,60.0,58.0,52.0,47.0,39.0,40.0,39.0,18.0
14,2000-01-15,30,17,58.0,58.0,47.0,39.0,22.0,39.0,18.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
8661,2023-09-18,68,60,90.0,82.0,77.0,77.0,77.0,60.0,60.0,59.0
8662,2023-09-19,72,58,84.0,82.0,77.0,77.0,68.0,60.0,60.0,60.0
8663,2023-09-20,74,58,82.0,79.0,77.0,72.0,72.0,60.0,60.0,58.0
8664,2023-09-21,71,57,82.0,77.0,74.0,74.0,74.0,60.0,58.0,58.0


In [297]:
# Date to datetime format

df['DATE'] = pd.to_datetime(df['DATE'])

In [298]:
df.dropna()

,DATE,TMAX,TMIN,TMAX_prev10,TMAX_prev7,TMAX_prev3,TMAX_prev2,TMAX_prev1,TMIN_prev3,TMIN_prev2,TMIN_prev1
10,2000-01-11,52,40,64.0,60.0,58.0,58.0,58.0,44.0,44.0,44.0
11,2000-01-12,47,39,64.0,58.0,58.0,58.0,52.0,44.0,44.0,40.0
12,2000-01-13,39,18,64.0,58.0,58.0,52.0,47.0,44.0,40.0,39.0
13,2000-01-14,22,13,60.0,58.0,52.0,47.0,39.0,40.0,39.0,18.0
14,2000-01-15,30,17,58.0,58.0,47.0,39.0,22.0,39.0,18.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
8661,2023-09-18,68,60,90.0,82.0,77.0,77.0,77.0,60.0,60.0,59.0
8662,2023-09-19,72,58,84.0,82.0,77.0,77.0,68.0,60.0,60.0,60.0
8663,2023-09-20,74,58,82.0,79.0,77.0,72.0,72.0,60.0,60.0,58.0
8664,2023-09-21,71,57,82.0,77.0,74.0,74.0,74.0,60.0,58.0,58.0


In [299]:
# Day and month features

df['day'] = df['DATE'].dt.day
df['month'] = df['DATE'].dt.month

In [300]:
df.sort_values(by = ['DATE'], inplace = True)

In [301]:
#df.set_index("DATE", inplace = True)
#df.drop([ 'DATE',], axis = 1, inplace = True)

In [302]:
df.isna().values.any()

True

In [303]:
df.fillna(-1, inplace = True)

In [304]:
# Removing the real time features , except TMAX since its label

df.drop([ 'TMIN',], axis = 1, inplace = True)


In [305]:
df.head()

,DATE,TMAX,TMAX_prev10,TMAX_prev7,TMAX_prev3,TMAX_prev2,TMAX_prev1,TMIN_prev3,TMIN_prev2,TMIN_prev1,day,month
0,2000-01-01,50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,1
1,2000-01-02,60,-1.0,-1.0,-1.0,-1.0,50.0,-1.0,-1.0,34.0,2,1
2,2000-01-03,64,-1.0,-1.0,-1.0,60.0,60.0,-1.0,43.0,43.0,3,1
3,2000-01-04,60,-1.0,-1.0,64.0,64.0,64.0,51.0,51.0,51.0,4,1
4,2000-01-05,47,-1.0,-1.0,64.0,64.0,60.0,51.0,51.0,46.0,5,1


In [306]:
train_df = df[df['DATE'] < '2023-08-01']
test_df = df[df['DATE'] >= '2023-08-01']


In [307]:
train_df.set_index("DATE", inplace = True)
test_df.set_index("DATE", inplace = True)


In [308]:
X_train, X_test, y_train, y_test = train_df.drop(['TMAX'], axis = 1).values, test_df.drop(['TMAX'], axis = 1).values, train_df['TMAX'].values, test_df['TMAX'].values 


In [309]:
model = XGBRegressor()
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [310]:
y_pred = model.predict(X_test)

In [311]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)

3.664945494453862

In [ ]:
pred_df = pd.DataFrame()
pred_df.index = test_df.index
pred_df['true_label'] = y_test
pred_df['pred_label'] = y_pred



In [ ]:
pred_df

,true_label,pred_label
DATE,,
2013-06-05,74,71.308144
2009-07-08,77,79.451904
2013-06-27,86,88.223953
2010-07-07,100,96.631401
2015-01-11,37,20.802273
...,...,...
2020-02-04,61,58.814465
2018-01-20,52,43.150597
2014-02-03,43,51.101540


In [313]:
import pickle
file_name = "Model/xgb_reg.pkl"

# save
pickle.dump(model, open(file_name, "wb"))


In [ ]:
########### LINEAR REGRESSION ##############

In [ ]:
lin_reg = LinearRegression()

In [ ]:
lin_reg.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lin_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)

5.33476882759863

In [ ]:
import pickle
file_name = "models/lin_reg.pkl"

# save
pickle.dump(model, open(file_name, "wb"))
